In [2]:
prompt = '''
你会得到一段通过ocr得到的关于流量卡的介绍信,你需要分析这些数据来了解这张卡,至少分析出以下数据：
1. 预充值金额: 【输出格式：100】(如:激活当月专属渠道首充50元享受优惠 - 50元, 激活强充100元话费享以下优惠 - 100元, 首充100后每月赠送180G通用流量 - 100元, 激活后七天内充值100元享受以下优惠不充值无法享受 - 100元等)
2. 初始价格:【输出格式：19】 (初始价格为第一个月(即首月)的价格,如:19元)
3. 最终价格:【输出格式：39】(3年后的价格,如: 39元)

请尽量保持准确性,不要偷懒,分析完后,将结果严格按以下列csv的格式发送给我，不需要给出原因：
{预充值金额},{初始价格},{最终价格}
如：100,19,39
'''

In [3]:
import requests
from lxml import etree


def get_172_product_info():
    url = 'https://haokawx.lot-ml.com/Product/Index/1'
    r = requests.get(url)
    html = etree.HTML(r.text)
    products = html.xpath('//ul[@class="fa"]/li')
    product_info = []
    for product in products:
        name = product.xpath('./a/div[@class="f1"]/dd/h1/text()')[0]
        if name not in [i['name'] for i in product_info]:
            product_info.append({
                'name': name,
                'pid': product.xpath('./a/@href')[0].split('&')[0].split('=')[-1],
                'tongyong': product.xpath('./a/div[@class="f1"]/dd/div[@class="b2"]/span[1]/text()')[0].split('\xa0')[-1],
                'dingxiang': product.xpath('./a/div[@class="f1"]/dd/div[@class="b2"]/span[2]/text()')[0].split('\xa0')[-1],
                'tonghua': product.xpath('./a/div[@class="f1"]/dd/div[@class="b2"]/span[3]/text()')[0].split('\xa0')[-1]
            })
    return product_info

products = get_172_product_info()

In [4]:
len(products)

84

In [5]:
def generate_pic(pid):
    url = f'https://haokawx.lot-ml.com/h5order/index?pudiD={pid}&userid=1'
    r = requests.get(url)
    html = etree.HTML(r.text)
    pic_url = html.xpath('//div[@class="view_8"]/div[@class="view_text"]/img/@src')[0]
    return pic_url

u = generate_pic('610')
u

'https://haokaapi.lot-ml.com/upload/ProductUpDateImage/20240507/9c589c458eb944e8b9a3879686c235d3.jpg'

In [6]:
API_KEY = ""
SECRET_KEY = ""

def ocr_pic(u):
    url = "https://aip.baidubce.com/rest/2.0/ocr/v1/general_basic?access_token=" + get_access_token()
    
    payload=f'url={u}&detect_direction=false&detect_language=false&paragraph=false&probability=false'
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Accept': 'application/json'
    }
    
    response = requests.request("POST", url, headers=headers, data=payload)
    
    words = response.json()['words_result']
    text = '\n'.join([i['words'] for i in words])
    return text
    

def get_access_token():
    """
    使用 AK，SK 生成鉴权签名（Access Token）
    :return: access_token，或是None(如果错误)
    """
    url = "https://aip.baidubce.com/oauth/2.0/token"
    params = {"grant_type": "client_credentials", "client_id": API_KEY, "client_secret": SECRET_KEY}
    return str(requests.post(url, params=params).json().get("access_token"))

ocr_result = ocr_pic(u)
ocr_result

'资费详情\n月费\n19元/月\n流量\n185G高速流量\n155G通用流量+30G定向流量\n语音\n0.1\n元/分钟\n优惠\n首月免费\n套餐详情介绍\n原套餐29元30G定向，首月月租按天折算，套餐内容按天\n折算\n激活当月专属渠道充50元享受优惠活动不充值无法享受\n1.激活首月赠送30元话费，激活当月有效，次月不结转。\n2.激活后京东处首充50减免的50元话费，50本金直接到账\n,减免的50元从次月起分5个月每月减免10元，到期可联\n系店铺继续参加存送活动。\n3.套餐激活且参加充值后72小时内到账155G通用流量，连\n续赠送24个月，赠送流量当月有效不可结转。\n4.综上所述：首月免费，2-6个月月租19元185G流量，到\n期联系店铺或者客服继续续约享受19元优惠。\n(首冲50元专属链接)\n(续约链接)\n温馨提示\n1.激活后请立即插入手机并立即拨打一次电话否则无法正常使\n用需要二次实名认证。\n2.归属地：归属地随机\n3.协议期：12个月。\n4.办理年龄：18周岁-60周岁。\n5.京东人员上门配送。入网预存不低于50元\n6,定向流量包含：（百度系、网易系、头条系）快手、腾讯视频\n优酷视频。更多定向软件联系客服。\n根据《刑法》第225条规定，个人擅自买卖已实名的电话卡属违\n法犯罪行为，请勿将你办理的手机卡转借、转租、买卖给他人，\n如被他人利用发生涉恐、诈骗、骚扰等非法违规行为，您将承担\n相应法律责任！\n申请注意事项：\n号卡类套餐地址填写学校是无法发货的，如果有学生需要请填写\n学校附近地址，信息内不能包含（学校，大学，学院，校区）等\n字样（尽量别填写菜鸟驿站，快递丰巢这类的）以防运营商审核\n不通过，建议写具体街道门牌号都行月底收到卡建议次月激活'

In [24]:
from openai import OpenAI

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key="",
    base_url="https://api.chatanywhere.cn/v1"
)

def chat(ocr_result):
    while True:
        try:
            completion = client.chat.completions.create(model='gpt-3.5-turbo', messages=[
                {
                    "role": "system",
                    "content": prompt
                },
                {
                    "role": "user",
                    "content": ocr_result
                }
            ])
            return completion.choices[0].message.content
        except:
            print('error occured, retrying...')
            pass

chat(ocr_result)


'100,39,39'

In [27]:
products = get_172_product_info()

f = open('result.csv', 'w')
f.write('名称,类型,预充值金额,初始价格,最终价格,通用流量,定向流量,通话时长\n')
for i in products:
    u = generate_pic(i['pid'])
    ocr_result = ocr_pic(u)
    result = chat(ocr_result).split(',')
    prestore, origin_price, final_price = result
    csv_result = f"{i['name']},{'长期卡' if origin_price == final_price else '短期卡'},{prestore},{origin_price},{final_price},{i['tongyong']},{i['dingxiang']},{i['tonghua']}"
    print(csv_result)
    f.write(csv_result + '\n')
f.close()

电信沧立卡新【19元235G 首年19】,长期卡,100,19,39,205G,30G,0分钟
电信沧久卡【19元185G+长期套餐】,长期卡,100,19,39,155G,30G,0分钟
电信沧蒙卡【19元255G】,长期卡,100,19,39,225G,30G,0分钟
联通梦明卡【29元210G+200分钟】,长期卡,100,29,39,210G,0G,200分钟
电信沧甘卡【19元235G+200分钟】,长期卡,100,19,39,205G,30G,200分钟
error occured, retrying...
